In [31]:
import glob
import pandas as pd

from PIL import Image
from PIL import ImageFilter
from PIL import ImageEnhance

from fastai.vision import *

### Create Train/Test/Eval

From the `../covers` directory generate train, test, and eval folders.

In [32]:
meta_path = "../metadata"
meta_paths = glob.glob(meta_path + "/*.csv")

# iteratively build meta_df
list_ = []
for path_ in meta_paths:
    df = pd.read_csv(path_, index_col=0, header=0)
    list_.append(df)

meta_df = pd.concat(list_).reset_index(drop=True)
meta_df.head()

,series,right,title,cover,metadata
0,X-Factor,(October 2012),X-Factor #242,./covers/X-Factor #242.jpg,"{'pencils': 'David Yardin (signed)', 'inks': '..."
1,X-Factor,(October 2012),X-Factor #241,./covers/X-Factor #241.jpg,"{'pencils': 'David Yardin (signed)', 'inks': '..."
2,X-Factor,(September 2012),X-Factor #239,./covers/X-Factor #239.jpg,"{'pencils': 'David Yardin (signed)', 'inks': '..."
3,X-Factor,(March 2012),X-Factor #230 [Nick Bradshaw Regenesis Gold Va...,./covers/X-Factor #230 [Nick Bradshaw Regenesi...,"{'pencils': 'Nick Bradshaw', 'inks': 'Nick Bra..."
4,X-Factor,(September 2012),X-Factor #240,./covers/X-Factor #240.jpg,"{'pencils': 'David Yardin (signed)', 'inks': '..."


In [33]:
cover_paths = os.listdir('../covers/')
cover_df = pd.DataFrame('./covers/' + pd.Series(cover_paths))
cover_df.columns = ['cover']

cover_df.head()

,cover
0,./covers/The Uncanny X-Men #202 [Canadian vari...
1,./covers/X-Men Classic #96 [Direct Edition].jpg
2,./covers/New X-Men #30.jpg
3,./covers/The X-Men #79.jpg
4,./covers/X-Men Classic #65 [Direct Edition].jpg


In [34]:
df = pd.merge(cover_df, meta_df, how='left').drop_duplicates().reset_index(drop=True)
df.head()

,cover,series,right,title,metadata
0,./covers/The Uncanny X-Men #202 [Canadian vari...,X-Men,NaN,The Uncanny X-Men #202 [Canadian variant],"{'pencils': 'John Romita, Jr. (signed)', 'inks..."
1,./covers/X-Men Classic #96 [Direct Edition].jpg,X-Men,(June 1994),X-Men Classic #96 [Direct Edition],"{'pencils': 'Matthew Ryan (signed)', 'inks': '..."
2,./covers/New X-Men #30.jpg,New X-Men,(November 2006),New X-Men #30,"{'pencils': 'Paco Medina [as PM!] (signed)', '..."
3,./covers/The X-Men #79.jpg,X-Men,(December 1972),The X-Men #79,"{'pencils': 'Gil Kane', 'inks': 'Frank Giacoia..."
4,./covers/X-Men Classic #65 [Direct Edition].jpg,X-Men,(November 1991),X-Men Classic #65 [Direct Edition],"{'pencils': 'Mike Mignola (signed)', 'inks': '..."


In [35]:
from ast import literal_eval

def get_characters_from_metadata_string(string):
    if string == "{}":
        return np.nan
    else:
        if "characters" in literal_eval(string).keys():
            return literal_eval(string)["characters"]
        else:
            return np.nan
    
df = pd.concat([df,pd.Series([get_characters_from_metadata_string(x) for x in df["metadata"]], name="characters")], axis=1)
df.head()

,cover,series,right,title,metadata,characters
0,./covers/The Uncanny X-Men #202 [Canadian vari...,X-Men,NaN,The Uncanny X-Men #202 [Canadian variant],"{'pencils': 'John Romita, Jr. (signed)', 'inks...",Magneto; Storm; Wolverine; Colossus; Rogue; Sh...
1,./covers/X-Men Classic #96 [Direct Edition].jpg,X-Men,(June 1994),X-Men Classic #96 [Direct Edition],"{'pencils': 'Matthew Ryan (signed)', 'inks': '...",X-Men [Rogue; Colossus; Nightcrawler]; Magus
2,./covers/New X-Men #30.jpg,New X-Men,(November 2006),New X-Men #30,"{'pencils': 'Paco Medina [as PM!] (signed)', '...",Hellion [Julian Keller]; X-23; Rockslide [Sant...
3,./covers/The X-Men #79.jpg,X-Men,(December 1972),The X-Men #79,"{'pencils': 'Gil Kane', 'inks': 'Frank Giacoia...",X-Men [Cyclops; Marvel Girl; Angel; Iceman; Be...
4,./covers/X-Men Classic #65 [Direct Edition].jpg,X-Men,(November 1991),X-Men Classic #65 [Direct Edition],"{'pencils': 'Mike Mignola (signed)', 'inks': '...",Profwssor X; Magneto; Baron Wolfgang von Strucker


In [53]:
def parse_characters_from_string(string):
    if type(string) is not str:
        return "None"
    else:
        return string.split("; ")
    
thingy = pd.Series([parse_characters_from_string(x) for x in df["characters"]])
s = thingy[1][0]
s

'X-Men [Rogue'

In [51]:
s[s.find("(")+1:s.find(")")]

AttributeError: 'list' object has no attribute 'find'

### Load Images

In [30]:
data = ImageDataBunch.from_folder(path='../covers/')

/Users/smacrae/github/cover-vision/.venv/lib/python3.6/site-packages/fastai/data_block.py:451: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")
/Users/smacrae/github/cover-vision/.venv/lib/python3.6/site-packages/fastai/data_block.py:454: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


IndexError: index 0 is out of bounds for axis 0 with size 0

### Metadata

In [ ]:
path =r'../metadata' # use your path
all_files = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in all_files:
    df = pd.read_csv(file_, index_col=0, header=0)
    list_.append(df)
df = pd.concat(list_).reset_index(drop=True)